### Chain of Responsability
#### [Referencia 1](https://refactoring.guru/pt-br/design-patterns/chain-of-responsibility)
#### [Referencia 2](https://www.youtube.com/watch?v=-zq_fKdkFrI&ab_channel=ProgramadorLhama)

#### O `Chain of Responsibility` é um padrão de projeto comportamental que permite que você passe pedidos por uma corrente de handlers. Ao receber um pedido, cada handler decide se processa o pedido ou o passa adiante para o próximo handler na corrente.
![img](https://refactoring.guru/images/patterns/content/chain-of-responsibility/chain-of-responsibility.png)

### Problema

##### Imagine que você está trabalhando em um sistema de encomendas online. Você quer restringir o acesso ao sistema para que apenas usuários autenticados possam criar pedidos. E também somente usuários que tem permissões administrativas devem ter acesso total a todos os pedidos.

##### Após um pouco de planejamento, você se dá conta que essas checagens devem ser feitas sequencialmente. A aplicação pode tentar autenticar um usuário ao sistema sempre que receber um pedido que contém as credenciais do usuário. Contudo, se essas credenciais não estão corretas e a autenticação falha, não há razão para continuar com outras checagens.
![img](https://refactoring.guru/images/patterns/diagrams/chain-of-responsibility/problem1-pt-br.png)

#### Durante os próximos meses você implementou diversas mais daquelas checagens sequenciais.

#### Um de seus colegas sugeriu que não é seguro passar dados brutos diretamente para o sistema de encomendas. Então você adicionou uma etapa adicional de validação para limpar os dados no pedido.

#### Mais tarde, alguém notou que o sistema é vulnerável à ataques de força bruta. Para evitar isso, você prontamente adicionou uma checagem que filtra repetidas falhas vindas do mesmo endereço de IP.

#### Outra pessoa sugeriu que você poderia agilizar o sistema se retornasse resultados de cache em pedidos repetidos contendo os mesmos dados. Portanto, você adicionou outra checagem que permite que o pedido passe através do sistema apenas se não há uma resposta adequada armazenada em cache.
![img](https://refactoring.guru/images/patterns/diagrams/chain-of-responsibility/problem2-pt-br.png)

### *Solução*

#### Como muitos outros padrões de projeto comportamental, o Chain of Responsibility `se baseia em transformar certos comportamentos em objetos solitários chamados handlers`. No nosso caso, `cada checagem devem ser extraída` para sua própria classe com um único método que faz a checagem. O pedido, junto com seus dados, é passado para esse método como um argumento.

#### `O padrão sugere que você ligue esses handlers em uma corrente.` Cada handler ligado tem um campo para armazenar uma referência ao próximo handler da corrente. Além de processar o pedido, handlers o passam adiante na corrente. O pedido viaja através da corrente até que todos os handlers tiveram uma chance de processá-lo.

#### E aqui está a melhor parte: um `handler pode decidir não passar o pedido adiante na corrente` e efetivamente parar qualquer futuro processamento.

#### Em nosso exemplo com sistema de encomendas, um handler realiza o processamento e então decide se passa o pedido adiante na corrente ou não. `Assumindo que o pedido contenha os dados adequados, todos os handlers podem executar seu comportamento principal, seja ele uma checagem de autenticação ou armazenamento em cache.`
![img](https://refactoring.guru/images/patterns/diagrams/chain-of-responsibility/solution1-pt-br.png)


#### Contudo, há uma abordagem ligeiramente diferente (e um tanto quanto canônica) na qual, ao receber o pedido, `um handler decide se ele pode processá-lo ou não`. Se ele pode, ele não passa o pedido adiante. Então é um handler que processa o pedido ou mais ninguém. Essa abordagem é muito comum quando lidando com eventos em pilha de elementos dentro de uma interface gráfica de usuário.

#### Por exemplo, quando um usuário clica um botão, `o evento se propaga através da corrente de elementos GUI que começam com aquele botão,` prossegue para seus contêineres (como planilhas ou painéis), e termina com a janela principal da aplicação. O evento é processado pelo primeiro elemento na corrente que é capaz de lidar com ele. Esse exemplo também é notável porque ele mostra que uma corrente pode sempre ser extraída de um objeto árvore.

#### *É crucial que todas as classes handler implementem a mesma interface. Cada handler concreto deve se importar apenas se o seguinte tem o método executar. Dessa maneira você pode compor correntes durante a execução, usando vários handlers sem acoplar seu código com suas classes concretas.*
![img](https://refactoring.guru/images/patterns/diagrams/chain-of-responsibility/solution2-pt-br.png)

#### Você acabou de comprar e instalar um novo hardware em seu computador. Como você é um geek, o computador tem diversos sistemas operacionais instalados. Você tenta ligar todos eles para ver se o hardware é suportado. O Windows detecta e ativa o hardware automaticamente. Contudo, seu amado Linux se recusa a trabalhar com o novo hardware. Com uma pequena ponta de esperança, você decide ligar para o número do suporte técnico escrito na caixa.

#### A primeira coisa que você ouve é uma voz robótica do outro lado. Ela sugere nove soluções populares para vários problemas, nenhum dos quais é relevante para seu caso. Após um tempo, a voz robótica conecta você com um operador de carne e osso.

#### Infelizmente, o operador não foi capaz de sugerir algo específico também. Ele continuava recitando longos protocolos do manual, se recusando a escutar seus comentários. Após escutar a frase “você tentou desligar e ligar o computador” pela décima vez, você exige ser conectado a um engenheiro.

#### Eventualmente o operador passa sua chamada para um dos engenheiros, que estava ansioso por contato humano já que estava sentado por horas em sua escura sala do servidor no subsolo de algum prédio. O engenheiro lhe diz onde baixar os drivers apropriados para seu novo hardware e como instalá-los no Linux. Finalmente, a solução! Você termina sua chamada, transbordando de alegria.
![img](https://refactoring.guru/images/patterns/content/chain-of-responsibility/chain-of-responsibility-comic-1-pt-br.png)

### Exemplo conceitual(Código)


In [24]:
from abc import abstractmethod, ABC

class PedidosOnline(ABC):
    @abstractmethod
    def validacao(self, **kwargs):
        ...
    
    @abstractmethod
    def execucao(self, *args):
        ...
      
        
class Autenticacao(PedidosOnline):
    def validacao(self, kwargs) -> bool:
        if kwargs.get('tipo', '') == 'validacao':
            return True
        return False
    
    def execucao(self) -> None:
        print('Processando validacao')
    


class PedidoPresente(PedidosOnline):
    def validacao(self, args) -> bool:
        if args.get('tipo', '') == 'presente':
            return True
        return False
    
    def execucao(self, *args) -> None:
        print("Embrulhando pedido para presente!")


class FechandoPedidoBrinquedo(PedidosOnline):
    def validacao(self, args) -> bool:
        if args.get('tipo', '') == 'brinquedo':
            return True
        return False
    
    def execucao(self) -> None:
        print('Fechando pedido de brinquedo!')
    
class LojaPedidosOnline:
    def __init__(self, fila_pedidos: list) -> None:
        self.fila_pedidos: list = fila_pedidos
        self.fila_execucao: list = [Autenticacao(), PedidoPresente(), 
                                    FechandoPedidoBrinquedo()]
        
    def validar_pedidos_qtde(self) -> bool:
        if len(self.fila_pedidos) == len(self.fila_execucao):
            return True
        return False
        
    def executar_pedidos(self) -> None:
        assert self.validar_pedidos_qtde(), 'Numero de pedidos passado incorreto!'
        for handle, pedido in zip(self.fila_execucao, self.fila_pedidos):
            if handle.validacao(pedido):
                handle.execucao()
            

pedidos = [{'tipo': 'validacao'}, {'tipo': 'presente'}, {'tipo': 'brinquedo'}]
loja = LojaPedidosOnline(pedidos)
loja.executar_pedidos()

pedidos = [{'tipo': ''}, {'tipo': ''}, {'tipo': 'brinquedo'}]
loja = LojaPedidosOnline(pedidos)
loja.executar_pedidos()

pedidos = [{'tipo': 'coisa'}, {'tipo': 'brinquedo'}]
loja = LojaPedidosOnline(pedidos)
loja.executar_pedidos()


Processando validacao
Embrulhando pedido para presente!
Fechando pedido de brinquedo!
Fechando pedido de brinquedo!


AssertionError: Numero de pedidos passado incorreto!